Este notebook está estruturado em células de análise, seguidas de células com as conclusões e encaminhamentos resultantes.

Descoberta 1 ...
* *Encaminhamento 1.1 ...*
* *Encaminhamento 1.2 ...*

Descoberta 2 ...
* *Encaminhamento 2.1 ...*
* *Encaminhamento 2.2 ...*

In [27]:
%matplotlib inline
%run ../scripts/toolbelt.py
import pandas as pd

In [31]:
df.Dt_Customer.dt.

0       2012-09-04
1       2014-03-08
2       2013-08-21
3       2014-02-10
4       2014-01-19
           ...    
2235    2013-06-13
2236    2014-06-10
2237    2014-01-25
2238    2014-01-24
2239    2012-10-15
Name: Dt_Customer, Length: 2240, dtype: object

In [30]:
df = pd.read_csv('../data/ml_project1_data.csv', parse_dates=True)

# Relatório gerado pelo pandas_profiling

**ID** é um numérico identificador único dos registros - considerar que é um dado que não carrega informação relevante para o problema
* *Utilizar como index do dataframe.*

**Marital_Status** possui valores que desconheço como valores significativos para o que entendo por "Marital Status": "Absurd", "YOLO", "Alone" (total de 7 registros)
* *Analisar os registros com esses valores, buscando verificar se há indícios de serem inválidos. Em caso positivo, remover do dataset.*

**Income:**
* possui como maior valor "666666", que chama a atenção tanto pelo padrão de repetição de dígitos, quanto por ser um valor bastante distante do segundo maior valor, "162397"
    * *Analisar o registro com esse valor, buscando verificar se há indícios de ser inválido. Em caso positivo, remover do dataset.*
* está com valor nulo em 24 registros
    * *Analisar os registros com **Income** null, buscando verificar se há indícios de serem inválidos. Em caso positivo, remover do dataset.*
    
**Kidhome** e **Teenhome** possuem distribuição de valores bastante similares (0 -> ~1.2k, 1 -> ~950, 2 -> ~50)
* *Analisar a distribuição conjunta dos dois atributos*

**Dt_Customer** é um campo de data
* *Extrair como atributos o ano e a diferença para 2015, que é o ano do maior valor de Dt_Customer + 1 (somado 1 para ter valor mínimo = 1 e permitir calcular médias de valores por ano de relacionamento).*

**Recency** - há um atributo referente ao último mês (**NumWebVisitsMonth**)
* *Gerar atributo indicando se **Recency** > 1 mês = 30*

Todas os atributos **Mnt** possuem uma distribuição com concentração maior nos menores valores. Ex: **MntWines**

<img src="../imgs/mntwine_histogram.png" />

* *Gerar atributo com o total gasto, somando todos atributos **Mnt**.*
* *Gerar, para cada atributo **Mnt**, um atributo contendo a proporção entre seu valor e o total gasto (definido no item acima).*
* *Gerar um atributo informando qual atributo **Mnt** tem maior valor entre todos **Mnt** do registro.*

**Num{sales_channel}Purchases** representam a quantidade de compras por canal de venda (não está especificado o período a que se referem esses dados, diferente das variáveis **Mnt**, que se referem aos últimos 2 anos)
* *Gerar atributo com o total de compras, somando todos atributos **Num{sales_channel}Purchases**.*
* *Gerar, para cada atributo **Num{sales_channel}Purchases**, um atributo contendo a proporção entre seu valor e o total de compras (definido no item acima).*
* *Gerar um atributo informando qual o atributo **Num{sales_channel}Purchases** tem maior valor entre todos **Num{sales_channel}Purchases** do registro.*

**NumDealsPurchases** representa a quantidade de compras usando descontos
* *Gerar atributo com a proporção entre **NumDealsPurchases** e o total de compras (ver notas de **Num{sales_channel}Purchases**)*.

**MntGoldProds** representa o total gasto com produtos *gold*
* *Gerar atributo com a proporção entre **MntGoldProds** e o total gasto

# Análise dos atributos

* Distribuição de valores
* Missing data
* Outliers
* Dados inválidos ou suspeitos


## Year_Birth

In [6]:
year_birth = df["Year_Birth"]

year_birth_desc = year_birth.describe(percentiles=[.1, .25, .5, .75, .9])

year_birth_desc

count    2240.000000
mean     1968.805804
std        11.984069
min      1893.000000
10%      1952.000000
25%      1959.000000
50%      1970.000000
75%      1977.000000
90%      1984.000000
max      1996.000000
Name: Year_Birth, dtype: float64

**O valor mínimo é suspeito (supondo que os dados foram gerados em 1996, o valor máximo, teríamos um cliente com ~103 anos).**
* *Utilizar um critério de detecção de outliers e analisar seus valores -> analisar os valores usando inter-quartile range.*

In [22]:
iqr_outliers(year_birth, kind='mild')

192    1900
239    1893
339    1899
Name: Year_Birth, dtype: int64

In [24]:
extreme_outliers = iqr_outliers(year_birth, kind='extreme')
_

192    1900
239    1893
339    1899
Name: Year_Birth, dtype: int64

**O valor mínimo é suspeito (supondo que os dados foram gerados em 1996, o valor máximo, teríamos um cliente com ~103 anos).**
* *Analisar os registros cujos anos são extreme outliers.*

In [26]:
df.loc[extreme_outliers.index].T

,192,239,339
ID,7829,11004,1150
Year_Birth,1900,1893,1899
Education,2n Cycle,2n Cycle,PhD
Marital_Status,Divorced,Single,Together
Income,36640.0,60182.0,83532.0
Kidhome,1,0,0
Teenhome,0,1,0
Dt_Customer,2013-09-26,2014-05-17,2013-09-26
Recency,99,23,36
MntWines,15,8,755
